# Week 8 
- Anant Sabharwal
- 200968138

## Importing necessary libraries

In [1]:
import gym
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
env = gym.make("FrozenLake-v1")

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.9/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.9/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


## Policy Iteration Function

In [4]:
def policy_iteration(policy, env, discount_factor, theta, max_iterations):
    '''Function to implement policy iteration
    '''

    num_s = env.observation_space.n
    num_act = env.action_space.n
    val_func = np.zeros(num_s)
    for i in range(max_iterations):
        while True:
            d = 0
            for s in range(num_s):
                old_val = val_func[s]
                act_p = policy[s]
                act_val = np.zeros(num_act)
                for a in range(num_act):
                    for p, next, reward, done in env.P[s][a]:
                        act_val[a] += p*(reward + discount_factor * val_func[next])
                val_func[s] = np.sum(act_p * act_val)
                d = max(d, np.abs(old_val - val_func[s]))
            if d < theta:
                break
        policy_stable = True
        for s in range(num_s):
            old_act = np.argmax(policy[s])
            act_val = np.zeros(num_act)
            for a in range(num_act):
                for p, next, reward, done in env.P[s][a]:
                    act_val[a] += p*(reward + discount_factor * val_func[next])
            best_act = np.argmax(act_val)
            if old_act != best_act:
                policy_stable = False
            policy[s] = np.eye(num_act)[best_act]
        if policy_stable:
            break  
    return policy


## Value Iteration Function

In [10]:
def value_iteration(env, discount_factor, theta, max_iterations):
    '''Function to implement value iteration
    '''
    num_s = env.observation_space.n
    num_act = env.action_space.n
    val_func = np.zeros(num_s)
    for i in range(max_iterations):
        d = 0
        for s in range(num_s):
            old_val = val_func[s]
            act_val = np.zeros(num_act)
            for a in range(num_act):
                for p, next, reward, done in env.P[s][a]:
                    act_val[a] += p*(reward + discount_factor * val_func[next])
            val_func[s] = np.max(act_val)
            d = max(d, np.abs(old_val - val_func[s]))
        if d < theta:
            break
    policy = np.zeros((num_s, num_act))
    for s in range(num_s):
        act_val = np.zeros(num_act)

        for a in range(num_act):
            for p, next, reward, done in env.P[s][a]:
                act_val[a] += p*(reward + discount_factor * val_func[next])

        best_act = np.argmax(act_val)
        policy[s][best_act] = 1.0    
    return policy

## Comparing Based on the number of episodes

In [11]:
def episodes(policy, env, episodes):
    '''Function to eun the episodes
    '''
    tot_reward = 0
    wins = 0
    for i in range(episodes):
        state = env.reset()
        ep_reward = 0
        flag = False
        while not flag:
            act = np.random.choice(env.action_space.n, p=policy[state])
            next, reward, flag, _ = env.step(act)
            ep_reward += reward
            state = next
        tot_reward += ep_reward
        if ep_reward == 1:
            wins += 1
    avg_reward = tot_reward/episodes
    return wins, avg_reward

In [12]:
#parameters
n_states = env.observation_space.n
n_actions = env.action_space.n

discount_factor = 0.9
theta = 1e-8
max_iterations = 1000
n_episode = 1000

In [13]:
# running policy iteration
policy = np.ones([n_states, n_actions]) / n_actions
opt_policy1 = policy_iteration(policy, env, discount_factor, theta, max_iterations)
wins1, avg_reward1 = episodes(opt_policy1, env, n_episode)

In [14]:
# running value iteration
opt_policy2 = value_iteration(env, discount_factor, theta, max_iterations)
wins2, avg_reward2 = episodes(opt_policy2, env, n_episode)

In [15]:
print("Policy Iteration:", f"Number of wins = {wins1}", f"Average Return = {avg_reward1}","", sep = '\n')
print("Value Iteration:", f"Number of wins = {wins2}", f"Average Return = {avg_reward2}","", sep = '\n')

Policy Iteration:
Number of wins = 722
Average Return = 0.722

Value Iteration:
Number of wins = 710
Average Return = 0.71



We observe that both methods, (a) Policy Iteration and (b) Value Iteration successfully converged to the optimal policy for the OpenAI FrozenLake-v1 environment, as seen by the high number of wins for each approach. 

However, we observe that for Policy Iteration, it has a higher number of wins and hence higher average reward. 

This implies that policy iteration may have started to converge to the optimal policy faster than Value iteration. 

This is applicable only when the discount factor is 0.9 and the results may change for different values of dicount factor.

Thus, from the results from this notebook, (at a discount factor of 0.9, max iterations of 1000) value iteration is the better choice to solve the FrozenLake-v1 enviorment. 